# Hopfield Network and RBM Comparison Experiments

This notebook compares the performance of Hopfield networks and Restricted Boltzmann Machines on the MNIST dataset, including comparisons of storage capacity, noise robustness, convergence speed, and feature learning.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as data
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd

# Add project root directory to path
sys.path.append('..')

# Import project modules
from src.hopfield.network import HopfieldNetwork
from src.hopfield.visualizer import HopfieldVisualizer
from src.hopfield.experiments import HopfieldExperiments
from src.boltzmann.rbm import RBM
from src.boltzmann.sampler import RBMSampler
from src.boltzmann.experiments import RBMExperiments
from src.utils.data_loader import MNISTLoader
from src.utils.preprocessing import binary_to_image
import config

## 1. Data Loading and Preprocessing

In [ ]:
# Create data loader
loader = MNISTLoader()

# Load MNIST data
train_data = loader.get_train_data(binary_values={0, 1})
test_data = loader.get_test_data(binary_values={0, 1})

# Get specific digit samples for Hopfield network
digits_to_use = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
hopfield_samples = []
hopfield_labels = []

for digit in digits_to_use:
    # Take 5 samples for each digit
    for i in range(5):
        sample, label = loader.get_specific_digit_sample(digit, binary_values={0, 1})
        hopfield_samples.append(sample)
        hopfield_labels.append(label)

# Convert to tensors
hopfield_samples = torch.stack(hopfield_samples)
hopfield_labels = torch.tensor(hopfield_labels)

# Create data loaders for RBM
train_loader = data.DataLoader(train_data, batch_size=config.RBM_CONFIG['batch_size'], shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=config.RBM_CONFIG['batch_size'], shuffle=False)

print(f"Hopfield samples shape: {hopfield_samples.shape}")
print(f"RBM training samples: {len(train_data)}")
print(f"RBM test samples: {len(test_data)}")

## 2. Hopfield Network Experiments

In [ ]:
# Create Hopfield network
n_neurons = hopfield_samples.shape[1]  # 784 (28*28)
hopfield_net = HopfieldNetwork(n_neurons=n_neurons)

# Train Hopfield network
print("Training Hopfield network...")
hopfield_train_time = time.time()
hopfield_net.train(hopfield_samples)
hopfield_train_time = time.time() - hopfield_train_time
print(f"Hopfield training time: {hopfield_train_time:.2f} seconds")

# Create visualizer
hopfield_viz = HopfieldVisualizer(hopfield_net)

# Create experiment manager
hopfield_exp = HopfieldExperiments(hopfield_net, hopfield_viz)

In [ ]:
# Test Hopfield network storage capacity
print("Testing Hopfield network storage capacity...")

# Test different numbers of patterns
pattern_counts = [5, 10, 20, 30, 40, 50]
hopfield_accuracies = []
hopfield_convergence_times = []

for count in pattern_counts:
    # Select specified number of patterns
    patterns = hopfield_samples[:count]
    
    # Create new network
    net = HopfieldNetwork(n_neurons=n_neurons)
    
    # Train network
    net.train(patterns)
    
    # Test recovery ability
    correct = 0
    total_time = 0
    
    for pattern in patterns:
        # Add small amount of noise
        noisy_pattern = pattern.clone()
        noise_indices = torch.randperm(noisy_pattern.size(0))[:int(0.1 * noisy_pattern.size(0))]
        noisy_pattern[noise_indices] = 1 - noisy_pattern[noise_indices]
        
        # Recover pattern
        start_time = time.time()
        recovered, converged = net.recover(noisy_pattern, max_iterations=100)
        total_time += time.time() - start_time
        
        # Check if correctly recovered
        if torch.allclose(recovered, pattern, atol=1e-5):
            correct += 1
    
    accuracy = correct / len(patterns)
    avg_time = total_time / len(patterns)
    
    hopfield_accuracies.append(accuracy)
    hopfield_convergence_times.append(avg_time)
    
    print(f"Patterns: {count}, Accuracy: {accuracy:.4f}, Avg. Convergence Time: {avg_time:.4f}s")

## 3. RBM Experiments

In [ ]:
# Create RBM
rbm = RBM(
    n_visible=config.RBM_CONFIG['n_visible'],
    n_hidden=config.RBM_CONFIG['n_hidden'],
    k=config.RBM_CONFIG['k'],
    learning_rate=config.RBM_CONFIG['learning_rate'],
    momentum=config.RBM_CONFIG['momentum'],
    weight_decay=config.RBM_CONFIG['weight_decay'],
    use_cuda=config.RBM_CONFIG['use_cuda']
)

# Train RBM
print("Training RBM...")
rbm_train_time = time.time()
rbm_errors = []
n_epochs = 20

for epoch in range(n_epochs):
    batch_errors = []
    
    for batch_idx, (data, _) in enumerate(train_loader):
        # Flatten data and binarize
        batch = data.view(data.size(0), -1)
        batch = (batch > 0.5).float()  # Binarize
        
        if rbm.use_cuda:
            batch = batch.cuda()
        
        # Train one batch
        error = rbm.train_batch(batch)
        batch_errors.append(error)
    
    # Calculate average error
    avg_error = np.mean(batch_errors)
    rbm_errors.append(avg_error)
    
    print(f"Epoch {epoch+1}/{n_epochs}, Error: {avg_error:.6f}")

rbm_train_time = time.time() - rbm_train_time
print(f"RBM training time: {rbm_train_time:.2f} seconds")

# Create sampler
rbm_sampler = RBMSampler(rbm)

# Create experiment manager
rbm_exp = RBMExperiments(rbm, rbm_sampler)

## 4. Storage Capacity Comparison

In [ ]:
# Compare storage capacities
print("Comparing storage capacities...")

# Hopfield network capacity test (already completed above)
hopfield_capacity = pattern_counts[np.argmax(hopfield_accuracies)]
print(f"Hopfield network effective capacity: ~{hopfield_capacity} patterns")

# RBM capacity test
rbm_pattern_counts = [50, 100, 200, 500, 1000, 2000, 5000]
rbm_accuracies = []

for count in rbm_pattern_counts:
    # Create data loader
    subset_data = data.Subset(train_data, range(count))
    subset_loader = data.DataLoader(subset_data, batch_size=64, shuffle=True)
    
    # Create new RBM
    test_rbm = RBM(
        n_visible=config.RBM_CONFIG['n_visible'],
        n_hidden=config.RBM_CONFIG['n_hidden'],
        k=config.RBM_CONFIG['k'],
        learning_rate=config.RBM_CONFIG['learning_rate'],
        momentum=config.RBM_CONFIG['momentum'],
        weight_decay=config.RBM_CONFIG['weight_decay'],
        use_cuda=config.RBM_CONFIG['use_cuda']
    )
    
    # Train RBM
    for epoch in range(10):  # Fewer training epochs to save time
        for batch_idx, (data, _) in enumerate(subset_loader):
            batch = data.view(data.size(0), -1)
            batch = (batch > 0.5).float()
            
            if test_rbm.use_cuda:
                batch = batch.cuda()
            
            test_rbm.train_batch(batch)
    
    # Test reconstruction ability
    test_subset = data.Subset(test_data, range(100))  # Use 100 test samples
    test_loader = data.DataLoader(test_subset, batch_size=64, shuffle=False)
    
    total_error = 0
    total_samples = 0
    
    for data, _ in test_loader:
        batch = data.view(data.size(0), -1)
        batch = (batch > 0.5).float()
        
        if test_rbm.use_cuda:
            batch = batch.cuda()
        
        # Calculate reconstruction error
        reconstructed = test_rbm.reconstruct(batch)
        error = torch.mean((batch - reconstructed) ** 2).item()
        
        total_error += error * batch.size(0)
        total_samples += batch.size(0)
    
    avg_error = total_error / total_samples
    accuracy = 1 - avg_error  # Convert error to accuracy
    rbm_accuracies.append(accuracy)
    
    print(f"Patterns: {count}, Reconstruction Accuracy: {accuracy:.4f}")

# Plot storage capacity comparison
fig, ax = plt.subplots(figsize=(10, 6))

# Plot Hopfield network results
ax.plot(pattern_counts, hopfield_accuracies, 'b-o', label='Hopfield Network', linewidth=2)

# Plot RBM results (using different x-axis)
ax2 = ax.twiny()
ax2.plot(rbm_pattern_counts, rbm_accuracies, 'r-s', label='RBM', linewidth=2)

# Set labels
ax.set_xlabel('Number of Patterns (Hopfield)', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Storage Capacity Comparison', fontsize=14)
ax.legend(loc='upper left')

ax2.set_xlabel('Number of Training Samples (RBM)', fontsize=12)
ax2.legend(loc='upper right')

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Noise Robustness Comparison

In [ ]:
# Compare noise robustness
print("Comparing noise robustness...")

# Test different noise levels
noise_levels = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
hopfield_noise_accuracies = []
rbm_noise_accuracies = []

# Select test samples
test_samples = hopfield_samples[:10]  # Use first 10 samples

for noise_level in noise_levels:
    # Test Hopfield network
    hopfield_correct = 0
    
    for sample in test_samples:
        # Add noise
        noisy_sample = sample.clone()
        noise_indices = torch.randperm(noisy_sample.size(0))[:int(noise_level * noisy_sample.size(0))]
        noisy_sample[noise_indices] = 1 - noisy_sample[noise_indices]
        
        # Recover pattern
        recovered, converged = hopfield_net.recover(noisy_sample, max_iterations=100)
        
        # Check if correctly recovered
        if torch.allclose(recovered, sample, atol=1e-5):
            hopfield_correct += 1
    
    hopfield_accuracy = hopfield_correct / len(test_samples)
    hopfield_noise_accuracies.append(hopfield_accuracy)
    
    # Test RBM
    rbm_total_error = 0
    
    for sample in test_samples:
        # Add noise
        noisy_sample = sample.clone()
        noise_indices = torch.randperm(noisy_sample.size(0))[:int(noise_level * noisy_sample.size(0))]
        noisy_sample[noise_indices] = 1 - noisy_sample[noise_indices]
        
        if rbm.use_cuda:
            noisy_sample = noisy_sample.cuda()
        
        # Reconstruct
        reconstructed = rbm.reconstruct(noisy_sample.unsqueeze(0))
        error = torch.mean((noisy_sample - reconstructed) ** 2).item()
        rbm_total_error += error
    
    avg_error = rbm_total_error / len(test_samples)
    rbm_accuracy = 1 - avg_error
    rbm_noise_accuracies.append(rbm_accuracy)
    
    print(f"Noise Level: {noise_level:.2f}, Hopfield Accuracy: {hopfield_accuracy:.4f}, RBM Accuracy: {rbm_accuracy:.4f}")

# Plot noise robustness comparison
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(noise_levels, hopfield_noise_accuracies, 'b-o', label='Hopfield Network', linewidth=2)
ax.plot(noise_levels, rbm_noise_accuracies, 'r-s', label='RBM', linewidth=2)

ax.set_xlabel('Noise Level', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Noise Robustness Comparison', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Convergence Speed Comparison

In [ ]:
# Compare convergence speeds
print("Comparing convergence speeds...")

# Hopfield network convergence test
hopfield_iterations = []
hopfield_times = []

for sample in test_samples[:5]:  # Use first 5 samples
    # Add noise
    noisy_sample = sample.clone()
    noise_indices = torch.randperm(noisy_sample.size(0))[:int(0.1 * noisy_sample.size(0))]
    noisy_sample[noise_indices] = 1 - noisy_sample[noise_indices]
    
    # Measure convergence time and iterations
    start_time = time.time()
    recovered, converged = hopfield_net.recover(noisy_sample, max_iterations=1000)
    end_time = time.time()
    
    hopfield_iterations.append(converged)
    hopfield_times.append(end_time - start_time)

# RBM reconstruction test
rbm_times = []
rbm_gibbs_steps = [10, 20, 50, 100, 200, 500]
rbm_errors = []

for steps in rbm_gibbs_steps:
    total_error = 0
    total_time = 0
    
    for sample in test_samples[:5]:
        # Add noise
        noisy_sample = sample.clone()
        noise_indices = torch.randperm(noisy_sample.size(0))[:int(0.1 * noisy_sample.size(0))]
        noisy_sample[noise_indices] = 1 - noisy_sample[noise_indices]
        
        if rbm.use_cuda:
            noisy_sample = noisy_sample.cuda()
        
        # Measure reconstruction time and error
        start_time = time.time()
        reconstructed = rbm.reconstruct(noisy_sample.unsqueeze(0), n_gibbs_steps=steps)
        end_time = time.time()
        
        error = torch.mean((noisy_sample - reconstructed) ** 2).item()
        
        total_error += error
        total_time += end_time - start_time
    
    avg_error = total_error / 5
    avg_time = total_time / 5
    
    rbm_errors.append(avg_error)
    rbm_times.append(avg_time)
    
    print(f"Gibbs Steps: {steps}, Avg. Error: {avg_error:.6f}, Avg. Time: {avg_time:.4f}s")

print(f"\nHopfield Network:")
print(f"Avg. Iterations: {np.mean(hopfield_iterations):.2f}")
print(f"Avg. Time: {np.mean(hopfield_times):.4f}s")

# Plot convergence speed comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Hopfield network iteration count distribution
ax1.hist(hopfield_iterations, bins=10, alpha=0.7, color='blue')
ax1.set_xlabel('Iterations to Convergence', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Hopfield Network Convergence', fontsize=14)
ax1.grid(True, alpha=0.3)

# RBM reconstruction error vs Gibbs steps
ax2.plot(rbm_gibbs_steps, rbm_errors, 'r-o', linewidth=2)
ax2.set_xlabel('Gibbs Steps', fontsize=12)
ax2.set_ylabel('Reconstruction Error', fontsize=12)
ax2.set_title('RBM Reconstruction Error vs Gibbs Steps', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Feature Learning Comparison

In [ ]:
# Compare feature learning capabilities
print("Comparing feature learning capabilities...")

# Hopfield network features (weight matrix)
hopfield_weights = hopfield_net.weights.detach().numpy()

# RBM features (weight matrix)
rbm_weights = rbm.get_weights().detach().cpu().numpy()

# Visualize weight patterns
fig, axes = plt.subplots(2, 10, figsize=(20, 4))

# Display Hopfield network weight patterns (select 10 neurons)
hopfield_indices = np.random.choice(hopfield_weights.shape[0], 10, replace=False)
for i, idx in enumerate(hopfield_indices):
    weight_img = hopfield_weights[idx].reshape(28, 28)
    axes[0, i].imshow(weight_img, cmap='seismic')
    axes[0, i].set_title(f'Neuron {idx}', fontsize=10)
    axes[0, i].axis('off')

# Display RBM weight patterns (select 10 hidden units)
rbm_indices = np.random.choice(rbm_weights.shape[1], 10, replace=False)
for i, idx in enumerate(rbm_indices):
    weight_img = rbm_weights[:, idx].reshape(28, 28)
    axes[1, i].imshow(weight_img, cmap='seismic')
    axes[1, i].set_title(f'Hidden {idx}', fontsize=10)
    axes[1, i].axis('off')

# Set titles
axes[0, 0].set_ylabel('Hopfield Network', fontsize=12)
axes[1, 0].set_ylabel('RBM', fontsize=12)

fig.suptitle('Weight Patterns Comparison', fontsize=16)
plt.tight_layout()
plt.show()

## 8. Comprehensive Performance Comparison

In [ ]:
# Create comprehensive performance comparison table
print("Creating comprehensive performance comparison...")

# Prepare data
comparison_data = {
    'Metric': ['Training Time (s)', 'Storage Capacity', 'Noise Robustness (10%)', 
               'Avg. Convergence Time (s)', 'Feature Learning'],
    'Hopfield Network': [
        f"{hopfield_train_time:.2f}",
        f"~{hopfield_capacity} patterns",
        f"{hopfield_noise_accuracies[1]:.4f}",
        f"{np.mean(hopfield_times):.4f}",
        "Hebbian Learning"
    ],
    'RBM': [
        f"{rbm_train_time:.2f}",
        "Large (scales with data)",
        f"{rbm_noise_accuracies[1]:.4f}",
        f"{rbm_times[2]:.4f}",
        "Contrastive Divergence"
    ]
}

# Create DataFrame
comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

# Save comparison results
comparison_df.to_csv('../results/comparison_results.csv', index=False)
print("Comparison results saved to '../results/comparison_results.csv'")

## 9. Recovery Example Visualization

In [ ]:
# Visualize recovery examples
print("Visualizing recovery examples...")

# Select test samples
test_samples = hopfield_samples[:5]
noise_level = 0.15

# Create figure
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for i, sample in enumerate(test_samples):
    # Original image
    original_img = binary_to_image(sample.numpy(), (28, 28))
    axes[0, i].imshow(original_img, cmap='binary')
    axes[0, i].set_title(f'Original {i}', fontsize=10)
    axes[0, i].axis('off')
    
    # Add noise
    noisy_sample = sample.clone()
    noise_indices = torch.randperm(noisy_sample.size(0))[:int(noise_level * noisy_sample.size(0))]
    noisy_sample[noise_indices] = 1 - noisy_sample[noise_indices]
    noisy_img = binary_to_image(noisy_sample.numpy(), (28, 28))
    axes[1, i].imshow(noisy_img, cmap='binary')
    axes[1, i].set_title(f'Noisy {i}', fontsize=10)
    axes[1, i].axis('off')
    
    # Hopfield network recovery
    hopfield_recovered, _ = hopfield_net.recover(noisy_sample, max_iterations=100)
    hopfield_img = binary_to_image(hopfield_recovered.numpy(), (28, 28))
    axes[2, i].imshow(hopfield_img, cmap='binary')
    axes[2, i].set_title(f'Hopfield {i}', fontsize=10)
    axes[2, i].axis('off')

# Set row titles
axes[0, 0].set_ylabel('Original', fontsize=12)
axes[1, 0].set_ylabel('Noisy', fontsize=12)
axes[2, 0].set_ylabel('Recovered', fontsize=12)

fig.suptitle('Recovery Examples Comparison', fontsize=16)
plt.tight_layout()
plt.savefig('../results/recovery_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Create RBM recovery examples
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for i, sample in enumerate(test_samples):
    # Original image
    original_img = binary_to_image(sample.numpy(), (28, 28))
    axes[0, i].imshow(original_img, cmap='binary')
    axes[0, i].set_title(f'Original {i}', fontsize=10)
    axes[0, i].axis('off')
    
    # Add noise
    noisy_sample = sample.clone()
    noise_indices = torch.randperm(noisy_sample.size(0))[:int(noise_level * noisy_sample.size(0))]
    noisy_sample[noise_indices] = 1 - noisy_sample[noise_indices]
    noisy_img = binary_to_image(noisy_sample.numpy(), (28, 28))
    axes[1, i].imshow(noisy_img, cmap='binary')
    axes[1, i].set_title(f'Noisy {i}', fontsize=10)
    axes[1, i].axis('off')
    
    # RBM recovery
    if rbm.use_cuda:
        noisy_sample_cuda = noisy_sample.cuda()
    else:
        noisy_sample_cuda = noisy_sample
    
    rbm_recovered = rbm.reconstruct(noisy_sample_cuda.unsqueeze(0), n_gibbs_steps=100)
    rbm_img = binary_to_image(rbm_recovered.detach().cpu().numpy(), (28, 28))[0]
    axes[2, i].imshow(rbm_img, cmap='binary')
    axes[2, i].set_title(f'RBM {i}', fontsize=10)
    axes[2, i].axis('off')

# Set row titles
axes[0, 0].set_ylabel('Original', fontsize=12)
axes[1, 0].set_ylabel('Noisy', fontsize=12)
axes[2, 0].set_ylabel('Recovered', fontsize=12)

fig.suptitle('RBM Recovery Examples', fontsize=16)
plt.tight_layout()
plt.savefig('../results/rbm_recovery_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Summary

This notebook compares the performance of Hopfield networks and Restricted Boltzmann Machines on the MNIST dataset, including:

1. **Storage Capacity**: Hopfield networks have limited storage capacity, while RBMs can handle large amounts of data
2. **Noise Robustness**: Both models have some noise robustness, but with different performance
3. **Convergence Speed**: Hopfield networks converge faster, while RBMs require more Gibbs sampling steps
4. **Feature Learning**: Hopfield networks use Hebbian learning, RBMs use contrastive divergence
5. **Training Time**: RBMs typically require longer training times

These comparisons help us understand the advantages and disadvantages of both models and their suitable application scenarios.